# Sample TIEGCM $\rho$ with Aqua
This notebook samples neutral density of the TIEGCM model along the Aqua satellite orbit.

    Aqua orbit data from this website: https://sscweb.gsfc.nasa.gov/cgi-bin/Locator.cgi

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

%matplotlib inline

import seaborn as sns
sns.set(style='ticks', context='talk')
import matplotlib.pyplot as plt


In [3]:
aqua_orbit_file = "../../Summer2019_data/aqua_orbit/aqua_data_2013_03_01_to_2013_03_04.csv"

aqua_data_raw = pd.read_csv(aqua_orbit_file, 
                         skiprows = 4 ,
                         names = ['date', 'X_geo','Y_geo','Z_geo','lat','lon','local_time_geo','radius'],
                         parse_dates = True,
                          )
# pd.to_datetime(aqua_data_raw['date'], format='%y/%m/%d %H:%M:%S' )#1362114000
aqua_ephemeris = pd.DataFrame(aqua_data_raw)


In [4]:
# print aqua_ephemeris['date'][1]
# aqua_ephemeris['date'] = pd.to_datetime(aqua_ephemeris['date'][1],format = ('%y/%m/%d %H:%M:%f'))
aqua_ephemeris['date'] = pd.to_datetime(aqua_ephemeris['date'],format = ('%y/%m/%d %H:%M:%f'))  
# print aqua_ephemeris['date']
new_date = aqua_ephemeris['date'].dt.strftime('%y-%m-%d %H:%M:%S')
# print new_date

# aqua_data_raw['date'] = aqua_data_raw['date'].dt.strftime('%m/%d/%Y')
# print (aqua_data_raw)

### Find altitude of satellite

In [5]:
R_e = 6378.137   # km 
altitude_aqua = aqua_ephemeris.radius - R_e

aqua_ephemeris['altitude'] = altitude_aqua 

# Import TIEGCM data

In [6]:
import pandas as pd
import numpy as np
from scipy.spatial import kdtree
from scipy.interpolate import griddata, LinearNDInterpolator
import scipy
from netCDF4 import Dataset
from plotly.offline import plot, iplot
import plotly.graph_objs as go
from plotly.graph_objs import Layout
import plotly
import matplotlib.pyplot as plt

plotly.offline.init_notebook_mode(connected=True)

# from tiegcm.tiegcm_original import TIEGCM as TIEGCM_original
# from tiegcm.tiegcm_original import Point3D as Point3D_original
# from tiegcm.tiegcm_original import Point4D as Point4D_original
# from tiegcm.tiegcm_original import Point4D as Point4D_original
# from tiegcm.tiegcm_original import Slice_key4D as Slice_key4D_original
# from tiegcm.tiegcm_original import ColumnSlice4D as ColumnSlice4D_original
# from tiegcm.tiegcm_original import ColumnSlice3D as ColumnSlice3D_original
# from tiegcm.util_original import average_longitude as average_longitude_original
# from tiegcm.util_original import *
# from tiegcm.tiegcm_original import Model_Manager as Model_Manager_o


from tiegcm.tiegcm import *
from tiegcm.util import *



### Create a function that we will use to extract the tiegcm data at any given point.

In [7]:
# density_model = []
# # tiegcm = TIEGCM('../data/TIEGCM_CCMC/my_stuff/s_003.nc')
# for i,idate in enumerate(aqua_ephemeris['date'][1:10]):
#         lon = np.round(np.array(aqua_ephemeris['lon'][i]) , decimals=2)
#         lat = np.round(np.array(aqua_ephemeris['lat'][i]) , decimals=2)
#         alt = np.round(np.array(aqua_ephemeris['altitude'][i]) , decimals=2)*1e5
#         idate = aqua_ephemeris['date'][i]
#         time = np.round(float(np.array( idate[9:11] )) , decimals=2)
#         print alt
# #         print lat
# #         print time
            
# #         density_model.append(tiegcm.density(lat, lon, alt, time ))


#### # Test the model manager

In [8]:
# directory = '../data/TIEGCM_CCMC/IRIDEA/main_2013_065_data-mult-fac-1p0.0/'

# model_test = Model_Manager_o(directory)

# xlon = 100
# xlat = 30
# xalt = 450*1e5
# time = 12

# density_mm = model_test.density(xlat, xlon, xalt,  pd.Timestamp('2013-03-01 1:00:02') )

# print model_test.files
# print model_test.file_times

In [9]:
def sample_TIEGCM(files, tiegcm_version):
    if tiegcm_version == "original":
        tiegcm = Model_Manager_o(files)
    elif tiegcm_version == "edited":
        tiegcm = Model_Manager(files)
#     new_date = pd.to_datetime(aqua_ephemeris['date'],format = ('%y/%m/%d %H:%M:%f'))  
    density_model = []
    title_y_string =  'density profile from ' + tiegcm_version + ' reader'
    title_string = tiegcm_version +' TIEGCM sampled using Aqua'

    

    for i,idate in enumerate(aqua_ephemeris['date'][0:50]):
        xlon = np.round(np.array(aqua_ephemeris['lon'][i]) , decimals=2)
        xlat = np.round(np.array(aqua_ephemeris['lat'][i])  , decimals=2)
        xalt = np.round(aqua_ephemeris['altitude'][i] , decimals=2) *1e5
        idate = aqua_ephemeris['date'][i]
#         time = new_date[50:100]
        time = new_date[i]
        print 'time = ', idate
        density_model.append(tiegcm.density(xlat, xlon, xalt,  idate ))

        
    plot_dens = go.Scatter(
                    x = aqua_ephemeris['date'][0:i],#plt_date ,
                    y = 1e3*np.array(density_model),
                      name= title_y_string, mode = 'markers',marker= dict(size =4) )

    data = [plot_dens]

    layout = go.Layout(
                    xaxis=dict( title = 'Date'),
                    yaxis=dict(type='log', autorange=True, exponentformat='power',
                           showexponent='all',title = 'Density [km]'), 
                    title = title_string)

#     fig = go.Figure(data=data, layout=layout)
#     iplot(fig)
   
    #plug into drag equation...?
    return plot_dens
    


In [10]:
    data = [
#            sample_TIEGCM('../data/TIEGCM_data/2013.03.01-2013.03.03_quiet/Cheyenne_2013.03.01.tiegcm.data/', 'original'),
           sample_TIEGCM('../../Summer2019_data/TIEGCM_data/2013.03.01-2013.03.03_quiet/CCMC_2013.03.01.tiegcm.data/', 'edited'),
    ]
#            sample_TIEGCM('../../Summer2019_data/TIEGCM_data/2013.03.01-2013.03.03_quiet/Cheyenne_2013.03.01.tiegcm.data/', 'edited')]
# data = [sample_TIEGCM('../data/TIEGCM_CCMC/s010.nc', 'original'),
#         sample_TIEGCM('../data/TIEGCM_CCMC/s010.nc', 'edited')]

layout = go.Layout(
                    xaxis=dict( title = 'Date'),
                    yaxis=dict(type='log', autorange=True, exponentformat='power',
                           showexponent='all',title = 'Density [km]'), 
                    title = "TIEGCM sampled using Aqua"
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)




initializing tiegcm with ../../Summer2019_data/TIEGCM_data/2013.03.01-2013.03.03_quiet/CCMC_2013.03.01.tiegcm.data/s001.nc
initializing tiegcm with ../../Summer2019_data/TIEGCM_data/2013.03.01-2013.03.03_quiet/CCMC_2013.03.01.tiegcm.data/s002.nc
initializing tiegcm with ../../Summer2019_data/TIEGCM_data/2013.03.01-2013.03.03_quiet/CCMC_2013.03.01.tiegcm.data/s003.nc
initializing tiegcm with ../../Summer2019_data/TIEGCM_data/2013.03.01-2013.03.03_quiet/CCMC_2013.03.01.tiegcm.data/s001.nc


tiegcm\tiegcm.py:213: UserWarning:

cannot be safely cast to variable data type



initializing tiegcm with ../../Summer2019_data/TIEGCM_data/2013.03.01-2013.03.03_quiet/CCMC_2013.03.01.tiegcm.data/s002.nc
initializing tiegcm with ../../Summer2019_data/TIEGCM_data/2013.03.01-2013.03.03_quiet/CCMC_2013.03.01.tiegcm.data/s003.nc
model manager initialized
current time (Timestamp('2013-03-01 00:20:00'), Timestamp('2013-03-01 08:00:00'))
time =  2013-03-01 00:00:00
time =  2013-03-01 00:01:00
time =  2013-03-01 00:02:00
time =  2013-03-01 00:03:00
time =  2013-03-01 00:04:00
time =  2013-03-01 00:05:00
time =  2013-03-01 00:06:00
time =  2013-03-01 00:07:00
time =  2013-03-01 00:08:00
time =  2013-03-01 00:09:00
time =  2013-03-01 00:10:00
time =  2013-03-01 00:11:00
time =  2013-03-01 00:12:00
time =  2013-03-01 00:13:00
time =  2013-03-01 00:14:00
time =  2013-03-01 00:15:00
time =  2013-03-01 00:16:00
time =  2013-03-01 00:17:00
time =  2013-03-01 00:18:00
time =  2013-03-01 00:19:00
time =  2013-03-01 00:20:00
time =  2013-03-01 00:21:00
time =  2013-03-01 00:22:00
ti